In [2]:
!pip install pytorch_lightning

import pandas as pd
from transformers import AutoModel, AutoTokenizer
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
import pytorch_lightning as pl
from torch.utils.data import DataLoader, TensorDataset
import torchmetrics
import math
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.metrics import f1_score, accuracy_score, classification_report
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.decomposition import PCA

%pip install camel-tools

from google.colab import drive
import os
# replace with your cameltools installation
os.environ['CAMELTOOLS_DATA'] = '/content/drive/MyDrive/camel_tools'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [3]:
!pip install Levenshtein
import Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 24.4 MB/s eta 0:00:00


In [4]:
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.disambig.bert import BERTUnfactoredDisambiguator
from pathlib import Path
S31_DB_PATH = Path('../data/disambig_db/calima-msa-s31.db')
S31_DB = MorphologyDB(S31_DB_PATH, 'a')
S31_AN = Analyzer(S31_DB, 'NOAN_ALL', cache_size=100000)
bert_disambig = BERTUnfactoredDisambiguator.pretrained('msa', top=1000, pretrained_cache = False)
bert_disambig._analyzer = S31_AN

Some weights of the model checkpoint at /content/drive/MyDrive/camel_tools/data/disambig_bert_unfactored/msa were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
import numpy as np
frag_train = pd.read_csv('../data/all_train_aligned.csv')
frag_dev = pd.read_csv('../data/all_dev_aligned.csv')
frag_test = pd.read_csv('../data/all_test_aligned.csv')


frag_train = frag_train[frag_train.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_dev = frag_dev[frag_dev.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_test = frag_test[frag_test.apply(lambda x: type(x['0']) == str, axis = 1)]

LEXICON DB


In [6]:
import pickle
with open('../data/lemma_db/quick_lemma_lookup.pkl', 'rb') as f:
    lemma_db = pickle.load(f)

def get_readability_level(analysis, token, oov_management, default_level = 0):
  lex = analysis['lex']
  pos = analysis['pos']
  result = lemma_db.get(lex)
  if result:
    if len(result) == 1:
      rl = result[0]['level']
    else:
      most_similar_element = None
      max_similarity = -1

      for element in result:
          similarity = Levenshtein.ratio(pos, element['pos'])
          if similarity > max_similarity:
              max_similarity = similarity
              most_similar_element = element

      rl = most_similar_element['level']
  else:
    rl = oov_management(token, default_level, analysis)
  return rl

Analysis tiebreaking

In [7]:
def sort_score(list_of_analyses):
  list_of_analyses.sort(key = lambda x: x.score, reverse = True)
  highest_score = list_of_analyses[0].score
  analyses_with_equal_score = [x for x in list_of_analyses
                                if x.score == highest_score]
  return analyses_with_equal_score


def sort_lexlogprob(list_of_analyses):
  list_of_analyses.sort(key = lambda x: x.analysis['lex_logprob'], reverse=True)
  highest_prob = list_of_analyses[0].analysis['lex_logprob']
  analyses_with_equal_prob = [x for x in list_of_analyses
                                if x.analysis['lex_logprob'] == highest_prob]
  return analyses_with_equal_prob

def sort_level(list_of_analyses):
  list_of_analyses.sort(key = lambda x: get_readability_level(x.analysis, '', default_level_oov, 9999))
  lowest_rl = get_readability_level(list_of_analyses[0].analysis, '', default_level_oov, 9999)
  analyses_with_equal_level = [x for x in list_of_analyses
                                    if get_readability_level(x.analysis, '', default_level_oov, 9999) == lowest_rl]
  return analyses_with_equal_level


def score_then_llp(list_of_analyses):
  list_of_analyses = sort_score(list_of_analyses)
  if len(list_of_analyses) == 1:
    return list_of_analyses[0].analysis

  list_of_analyses = sort_lexlogprob(list_of_analyses)
  return list_of_analyses[0].analysis


def score_then_level(list_of_analyses):
  list_of_analyses = sort_score(list_of_analyses)
  if len(list_of_analyses) == 1:
    return list_of_analyses[0].analysis

  list_of_analyses = sort_level(list_of_analyses)
  return list_of_analyses[0].analysis

def score_then_llp_then_level(list_of_analyses):
  list_of_analyses = sort_score(list_of_analyses)
  if len(list_of_analyses) == 1:
    return list_of_analyses[0].analysis

  list_of_analyses = sort_lexlogprob(list_of_analyses)
  if len(list_of_analyses) == 1:
    return list_of_analyses[0].analysis

  list_of_analyses = sort_level(list_of_analyses)
  return list_of_analyses[0].analysis


def score_then_level_then_llp(list_of_analyses):
  list_of_analyses = sort_score(list_of_analyses)
  if len(list_of_analyses) == 1:
    return list_of_analyses[0].analysis

  list_of_analyses = sort_level(list_of_analyses)
  if len(list_of_analyses) == 1:
    return list_of_analyses[0].analysis

  list_of_analyses = sort_lexlogprob(list_of_analyses)
  return list_of_analyses[0].analysis




OOV Management

In [8]:
with open('../data/levels_db/mle_max_aligned_model.pkl', 'rb') as f:
  mle_model = pickle.load(f)

with open('../data/freq/freq_token_db.pkl', 'rb') as f:
  freq_backoff = pickle.load(f)

def default_level_oov(word, level, analysis):
  return level

def default_level_nounprop(word, level, analysis):
  if analysis['pos'] == 'NOUN_PROP':
    return level
  else:
    return 5

def mle_oov(word, level, analysis):
  return mle_model.get(word, level)

def freq_oov(word, level, analysis):
  return freq_backoff.get(word, level)

def mle_nounprop(word, level, analysis):
  if analysis['pos'] == 'NOUN_PROP':
    return level
  else:
    return mle_oov(word, level, analysis)





Inference pipeline

In [9]:
def lexicon_levels_pipeline(fragment, tiebreak, oov_management):
  tokens = [t.split('#')[0] for t in fragment.split(' ')]
  analyses = [token.analyses for token in bert_disambig.disambiguate(tokens)]
  picked_lexs = [tiebreak(analysis) for analysis in analyses]
  levels = [
      get_readability_level(analysis, token, oov_management) for analysis, token in zip(picked_lexs, tokens)
  ]
  levels = [a if a > 3 else 3 for a in levels]
  return levels

In [10]:
def get_gt_levels(fragment):
  return [int(t.split('#')[1]) for t in fragment.split(' ')]

gt_levels = np.concatenate([get_gt_levels(f) for f in frag_test['0']])

In [11]:
# deleted: aggregation, resolve_zeroes_if_needed.

exp_1 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_llp, default_level_oov) for fragment in (frag_test['0'])])
exp_2 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_llp, mle_oov) for fragment in (frag_test['0'])])
exp_4 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_llp, default_level_nounprop) for fragment in (frag_test['0'])])
exp_5 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_llp, mle_nounprop) for fragment in (frag_test['0'])])

# score then level
exp_6 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_level, default_level_oov) for fragment in (frag_test['0'])])
exp_7 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_level, mle_oov) for fragment in (frag_test['0'])])
exp_9 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_level, default_level_nounprop) for fragment in (frag_test['0'])])
exp_10 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_level, mle_nounprop) for fragment in (frag_test['0'])])

# prob then level
exp_11 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_llp_then_level, default_level_oov) for fragment in (frag_test['0'])])
exp_12 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_llp_then_level, mle_oov) for fragment in (frag_test['0'])])
exp_14 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_llp_then_level, default_level_nounprop) for fragment in (frag_test['0'])])
exp_15 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_llp_then_level, mle_nounprop) for fragment in (frag_test['0'])])

# level then prob
exp_16 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_level_then_llp, default_level_oov) for fragment in (frag_test['0'])])
exp_17 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_level_then_llp, mle_oov) for fragment in (frag_test['0'])])
exp_19 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_level_then_llp, default_level_nounprop) for fragment in (frag_test['0'])])
exp_20 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_level_then_llp, mle_nounprop) for fragment in (frag_test['0'])])

# with mle backoff
exp_21 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_llp, freq_oov) for fragment in (frag_test['0'])])
exp_22 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_level, freq_oov) for fragment in (frag_test['0'])])
exp_23 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_llp_then_level, freq_oov) for fragment in (frag_test['0'])])
exp_24 = np.concatenate([lexicon_levels_pipeline(fragment, score_then_level_then_llp, freq_oov) for fragment in (frag_test['0'])])

In [ ]:
from sklearn.metrics import classification_report
print('==============================exp_1===============')
print(classification_report(gt_levels, exp_1))
print('==============================exp_2===============')
print(classification_report(gt_levels, exp_2))
print('==============================exp_4===============')
print(classification_report(gt_levels, exp_4))
print('==============================exp_5===============')
print(classification_report(gt_levels, exp_5))
print('==============================exp_6===============')
print(classification_report(gt_levels, exp_6))
print('==============================exp_7===============')
print(classification_report(gt_levels, exp_7))
print('==============================exp_9===============')
print(classification_report(gt_levels, exp_9))
print('==============================exp_10===============')
print(classification_report(gt_levels, exp_10))
print('==============================exp_11===============')
print(classification_report(gt_levels, exp_11))
print('==============================exp_12===============')
print(classification_report(gt_levels, exp_12))
print('==============================exp_14===============')
print(classification_report(gt_levels, exp_14))
print('==============================exp_15===============')
print(classification_report(gt_levels, exp_15))
print('==============================exp_16===============')
print(classification_report(gt_levels, exp_16))
print('==============================exp_17===============')
print(classification_report(gt_levels, exp_17))
print('==============================exp_19===============')
print(classification_report(gt_levels, exp_19))
print('==============================exp_20===============')
print(classification_report(gt_levels, exp_20))
print('==============================exp_21===============')
print(classification_report(gt_levels, exp_21))
print('==============================exp_22===============')
print(classification_report(gt_levels, exp_22))
print('==============================exp_23===============')
print(classification_report(gt_levels, exp_23))
print('==============================exp_24===============')
print(classification_report(gt_levels, exp_24))